In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import glob
import os
import requests
import time
import warnings

warnings.simplefilter('ignore', FutureWarning)

from collections import defaultdict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm.auto import tqdm

import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary

import torchvision
from torchvision import transforms

from IPython.display import clear_output
%matplotlib inline

sns.set(font_scale=1.2)
# sns.set_style(style='whitegrid')
device_num = 0
#torch.cuda.set_device(device_num)#####################################################
from google.colab import drive
import shutil
import random


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
tqdm.pandas()

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shutil.copy('/content/drive/MyDrive/CURS_22_DF/19_12_scale_df_signals.pkl', '/content/',  )

'/content/19_12_scale_df_signals.pkl'

In [ ]:
df = pd.read_pickle('19_12_scale_df_signals.pkl')
df

,type_sig,signal_IQ
0,PSK_2,"[(0.7059837322161113-0.7082280493218339j), (-0..."
0,PSK_2,"[(-0.8728823514927556-0.487930733252659j), (-0..."
0,PSK_2,"[(0.44848191175040925+0.8937919080147785j), (0..."
0,PSK_2,"[(0.5286214672583299-0.8488576702567107j), (-0..."
0,PSK_2,"[(0.8079779931435035-0.5892126633023059j), (-0..."
...,...,...
0,QAM_256,"[(-0.03005561565198769-0.059507179501505844j),..."
0,QAM_256,"[(-0.6602920525129635-0.41581641896046706j), (..."
0,QAM_256,"[(-0.2235837042777474+0.3636379372448136j), (-..."
0,QAM_256,"[(0.42779005460406133-0.5156399500336869j), (0..."


In [ ]:
def noise( x, N):
        snr_dB=N
        snr = 10**(snr_dB/10)
        sigmaN2 = 1/snr

        # generate AWGN
        n = (sigmaN2**0.5)*(np.random.randn(len(x)) + 1j*np.random.randn(len(x)))
        return x + n

def make_noise(df, N):
    df_w = df.copy()
    df_w['signal_IQ'] = df_w['signal_IQ'].apply(lambda x: noise(x, N))
    return df_w

def moment(x,p,q):
    return np.mean(x**(p-q) * x.conjugate()**q)
def cumulant(x,p,q):
        if p == 2:
            return moment(x,p,q)
        elif p == 4:
            if q == 0:
                return moment(x,4,0) - 3 * moment(x,2,0) **2
            elif q == 1:
                return moment(x,4,1) - 3 * moment(x,2,1) **2
            return moment(x,4,2) - moment(x,2,0).real**2 - 2 * moment(x,2,1) **2

def make_mom_cum(df):
    df_out = df.copy()
    df_cols_name = []
    moments_num = [[1, 0], [2, 0], [2, 1], [3, 0], [3, 1], [3, 2], [4, 0]]
    for num in (moments_num):
        df_cols_name.append(f'moment_{num[0]}_{num[1]}')
        df_out[df_cols_name[-1]] = df_out.signal_IQ.apply(lambda x: moment(x,num[0], num[1]))
    cumulants_num = [(2,0),(2,1),(4,0),(4,1),(4,2)]
    for num in (cumulants_num):
        df_cols_name.append(f'cumulants_{num[0]}_{num[1]}')
        df_out[df_cols_name[-1]] = df_out.signal_IQ.apply(lambda x: cumulant(x,num[0], num[1]))

    return df_out

In [ ]:
list_SNR = sorted(list( set(list(range(-30,31,5)) + list(range(-10,11,1))) ))
list_SNR = sorted(list(range(-20,21,2)))
print(len(list_SNR))
list_SNR

21


[-20,
 -18,
 -16,
 -14,
 -12,
 -10,
 -8,
 -6,
 -4,
 -2,
 0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20]

In [ ]:
%%time
dict_classes = {}
for i in tqdm(list_SNR):
    noise_sig = make_noise(df,i)
    df_train = make_mom_cum(noise_sig)
    X = df_train.drop(['type_sig','signal_IQ'],axis =1)
    y = df_train['type_sig']
    le = preprocessing.LabelEncoder()
    y1 = le.fit_transform(y)
    dict_classes[i] = {}
    dict_classes[i]['X'] =  X
    dict_classes[i]['y'] = y

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 3min 44s, sys: 4.45 s, total: 3min 48s
Wall time: 3min 50s


In [ ]:
models = {'CatBoostClassifier':CatBoostClassifier(silent=True, iterations = 300),#,task_type="GPU"),
          'RandomForestClassifier':RandomForestClassifier(),
          'ExtraTreesClassifier':ExtraTreesClassifier(),
          'DecisionTreeClassifier':DecisionTreeClassifier(),
          #'MultinomialNB':MultinomialNB(),
          'SVC':SVC(),
          'LogisticRegression':LogisticRegression(),
          'KNeighborsClassifier':KNeighborsClassifier()
}

In [ ]:
df_cv = pd.DataFrame()
for key, model in tqdm(models.items() ):

    df_add = pd.DataFrame()
    df_add.loc[key,'model'] = key
    for key_2 in tqdm(dict_classes.keys()):
        X = dict_classes[key_2]['X'].applymap(lambda x: (x).real )
        y = dict_classes[key_2]['y']
        le = preprocessing.LabelEncoder()
        y1 = le.fit_transform(y)

        df_add.loc[key,key_2] = cross_val_score(model, X, y1, cv = 5).mean()
    df_cv = pd.concat([df_cv, df_add])

In [ ]:
plot = df_cv.drop('model',axis = 1).T.sort_index().reset_index()
plot

,index,CatBoostClassifier,RandomForestClassifier,ExtraTreesClassifier,DecisionTreeClassifier,SVC,LogisticRegression,KNeighborsClassifier
0,-20,0.123250,0.127417,0.127250,0.121333,0.124875,0.122208,0.121958
1,-18,0.127958,0.126500,0.127208,0.125167,0.131417,0.122792,0.125375
2,-16,0.131333,0.129833,0.127000,0.124542,0.133417,0.126250,0.120333
3,-14,0.137625,0.131167,0.127417,0.127208,0.140500,0.126417,0.125708
4,-12,0.143333,0.141375,0.140125,0.130208,0.144083,0.125625,0.133125
5,-10,0.178083,0.175333,0.172250,0.148000,0.159917,0.147708,0.140083
6,-8,0.230875,0.218542,0.216750,0.174375,0.182250,0.179292,0.161333
7,-6,0.300958,0.270125,0.267833,0.230708,0.204458,0.190917,0.207583
8,-4,0.385542,0.351083,0.337708,0.310417,0.250333,0.234875,0.278167
9,-2,0.478542,0.456417,0.442542,0.408042,0.337708,0.282667,0.375000


In [ ]:
plot.to_csv('CV_res.csv')

In [ ]:
accuracy_class = plot.rename(columns = {'index':'SNR'})

In [ ]:
df_long = pd.melt(accuracy_class,['SNR'],var_name = 'model', value_name = 'accuracy')
#df_long

In [ ]:
df_long

,SNR,model,accuracy
0,-20,CatBoostClassifier,0.123250
1,-18,CatBoostClassifier,0.127958
2,-16,CatBoostClassifier,0.131333
3,-14,CatBoostClassifier,0.137625
4,-12,CatBoostClassifier,0.143333
...,...,...,...
142,12,KNeighborsClassifier,0.950708
143,14,KNeighborsClassifier,0.966083
144,16,KNeighborsClassifier,0.977917
145,18,KNeighborsClassifier,0.983750


In [ ]:
import plotly.express as px

fig = px.line(df_long, x = 'SNR', y = 'accuracy', color = 'model', symbol="model",
              title="Точность классификации CatBoostClassifier",)
fig.show()

#DL

In [ ]:
device = f"cuda:{device_num}" if torch.cuda.is_available() else "cpu"
#device = "cpu"
device

'cuda:0'

In [ ]:
class digitsPerceptron(nn.Module):
    acrivete_func = nn.Softsign()
    def __init__(self, input_dim=1024, hidden_dim=100):
        super().__init__()
        self.n_classes = 8
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            self.acrivete_func,
            nn.Linear(hidden_dim, hidden_dim),
            self.acrivete_func,
            # nn.Linear(hidden_dim, hidden_dim),
            # self.acrivete_func,
            nn.Linear(hidden_dim, self.n_classes),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        return self.net(x)

model = digitsPerceptron(
        #input_dim, 
        #hidden_dim
    ).to(device)

In [ ]:
acc = {}
for key in tqdm(dict_classes.keys()):
    X = dict_classes[key]['X']
    y = dict_classes[key]['y']
    le = preprocessing.LabelEncoder()
    y1 = le.fit_transform(y)
    X_train_in, X_test_in, y_train_in, y_test_in = train_test_split(
        X, y1, test_size=0.2, random_state=21,
        shuffle=True, stratify=y
    )

    model = digitsPerceptron(            
            input_dim=len(X.columns), 
            hidden_dim=100
        ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    X_train_in.reset_index(drop=True,inplace = True)
    X_test_in.reset_index(drop=True,inplace = True)     
    X_train = torch.tensor(X_train_in.values, dtype=torch.float).to(device)
    y_train = torch.tensor(y_train_in, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test_in.values, dtype=torch.float).to(device)
    y_test = torch.tensor(y_test_in, dtype=torch.long).to(device)

    EPOCHS  = 300
    loss_list = np.zeros((EPOCHS,))
    accuracy_list = np.zeros((EPOCHS,))
    for epoch in tqdm(range(EPOCHS)):
        model.train()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        loss_list[epoch] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.float)
            accuracy_list[epoch] = correct.mean().item() 
    model.eval()

    with torch.no_grad():
        y_pred = np.argmax(model(X_test).detach().cpu().numpy(), axis=1)
    acc[key] = {}
    
    acc[key]['accuracy_score'] = accuracy_score(y_test_in, y_pred)
    acc[key]['classification_report'] = classification_report(y_test_in, y_pred, output_dict = True)
    acc[key]['confusion_matrix'] = confusion_matrix(y_test_in, y_pred)


  0%|          | 0/21 [00:00<?, ?it/s]

<ipython-input-14-950c7ea4e821>:20: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:250.)
  X_train = torch.tensor(X_train_in.values, dtype=torch.float).to(device)


  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
arr = []
for key in tqdm(acc.keys()):
    matrix = acc[key]['confusion_matrix']
    arr.append((matrix.diagonal()/matrix.sum(axis=1)))

In [ ]:
accuracy_class = pd.DataFrame(arr, index = acc.keys(), columns = le.inverse_transform([0,1,2,3,4,5,6,7]))

In [ ]:
accuracy_class

In [ ]:
arr = []
for key in tqdm(acc.keys()):
    matrix = acc[key]['confusion_matrix']
    arr.append((matrix.diagonal()/matrix.sum(axis=1)))

In [ ]:
arr_accuracy_score = []
for key in tqdm(acc.keys()):
    arr_accuracy_score.append(acc[key]['accuracy_score'])

  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
arr_accuracy_score

[0.12104166666666667,
 0.12541666666666668,
 0.12666666666666668,
 0.13208333333333333,
 0.14291666666666666,
 0.16958333333333334,
 0.21291666666666667,
 0.300625,
 0.3416666666666667,
 0.461875,
 0.49125,
 0.5845833333333333,
 0.6810416666666667,
 0.7475,
 0.8295833333333333,
 0.77875,
 0.881875,
 0.8460416666666667,
 0.8633333333333333,
 0.9091666666666667,
 0.9441666666666667]

In [ ]:
pd.DataFrame(arr_accuracy_score).to_csv('neiro.csv')

#ACC

In [ ]:
arr_accuracy_score = {}
for key in tqdm(acc.keys()):
    arr_accuracy_score[key] = acc[key]['accuracy_score']

  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
pd.DataFrame(arr_accuracy_score,index = ['neiro']).T.to_csv('neiro_2.csv')

#catboost

In [ ]:
acc = {}
for key in tqdm(dict_classes.keys()):
    X = dict_classes[key]['X']
    y = dict_classes[key]['y']
    le = preprocessing.LabelEncoder()
    y1 = le.fit_transform(y)
    X_train_in, X_test_in, y_train_in, y_test_in = train_test_split(
        X, y1, test_size=0.2, random_state=21,
        shuffle=True, stratify=y
    )

    cat = CatBoostClassifier(iterations = 800)
    cat.fit(X_train_in,y_train_in,
        use_best_model = True,
        silent = True,
        eval_set = (X_test_in,y_test_in))
    acc[key] = {}
    y_pred = cat.predict(X_test_in)
    acc[key]['accuracy_score'] = accuracy_score(y_test_in, y_pred)
    acc[key]['classification_report'] = classification_report(y_test_in, y_pred, output_dict = True)
    acc[key]['confusion_matrix'] = confusion_matrix(y_test_in, y_pred)



NameError: ignored